In [1]:
import os

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_2-part_error.xlsx']

In [2]:
file = list_file[0]

file

'crawler_2-part_error.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/1.url/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：3


['https://www.1aauto.com/1978-81-chevrolet-pontiac-roofrail-weatherstrip-seal-2-piece-set-metro-moulded-parts-rr-5020/i/1awsr00095',
 'https://www.1aauto.com/ford-front-driver-side-door-weatherstrip-seal-fairchild-automotive-f3004/i/1awsd00176',
 'https://www.1aauto.com/chevrolet-gmc-olds-inner-and-outer-window-sweep-4-piece-set-fairchild-automotive-kg2006/i/1awss00115']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = input_.loc[a, 'Url']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            title = html.xpath('//head/title/text()')[0].strip()
            with open('./part.html', 'w', encoding='UTF-8') as file:
                file.write(str(soup))
            break
    except KeyboardInterrupt:
        break
    except:
        continue

title

1


'1978-81 Chevrolet Pontiac Roofrail Weatherstrip Seal 2 Piece Set Metro Moulded Parts RR 5020'

In [8]:
type_ = html.xpath('//span[@itemprop="name"]')[-1].xpath('./text()')[0].strip()

type_

'Weatherstripping'

In [9]:
tag = html.xpath('//p[@class="product-title"]/text()')[0].strip() if html.xpath('//p[@class="product-title"]/text()') else ''

tag

'Replaces'

In [10]:
dict_src, list_src = {}, html.xpath('//div[@class="oneaCarousel onea-carousel-container onea-carousel-vertical"]/ul[@class="oneaCarouselBody onea-carousel-body invisible-scroll-y"]/li/img[@class="img-sq-s lazy-img lazyOnReady carouselDisplaySelect"]/@data-replacement')
for i, src in enumerate(list_src):
    dict_src[str(i)] = f'''https:{src.strip()}'''

dict_src

{'0': 'https://cdn.1aauto.com/partimage/WSR/1AWSR00095/2187fb6f954c4e6cb9c4287963975d64_490.jpg'}

In [11]:
list_div, dict_count = html.xpath('//div[@id="product-description-tab"]/div/ul/li'), {}
for div in list_div:
    list_text = [text.strip().removesuffix(':').strip() for text in div.xpath('./span/text()')]
    if len(list_text) > 1:
        if list_text[0] in dict_count:
            dict_count[list_text[0]] += 1
        else:
            dict_count[list_text[0]] = 1

list_specification = html.xpath('//div[@id="product-description-tab"]/specifications/specification')
for specification in list_specification:
    key = specification.xpath('./text()')[0].strip()
    if key in dict_count:
        dict_count[key] += 1
    else:
        dict_count[key] = 1

list_p = html.xpath('//div[@id="product-description-tab"]/p')
for p in list_p:
    if p.xpath('./strong/text()'):
        key = p.xpath('./strong/text()')[0].strip().removesuffix(':').strip()
        if key in ['Item Condition', 'WARNING']:
            if key in dict_count:
                dict_count[key] += 1
            else:
                dict_count[key] = 1

dict_count

{'Quantity': 1, 'Item Condition': 1, 'WARNING': 1}

In [12]:
i, dict_description, dict_no = 0, {}, {}
for div in list_div:
    list_text = [text.strip().removesuffix(':').strip() for text in div.xpath('./span/text()')]
    if len(list_text) == 2:
        if dict_count[list_text[0]] == 1:
            dict_description[str(i)] = {list_text[0]: list_text[1]}
        else:
            if list_text[0] in dict_no:
                dict_no[list_text[0]] += 1
            else:
                dict_no[list_text[0]] = 1
            dict_description[str(i)] = {f'{list_text[0]}-{dict_no[list_text[0]]}': list_text[1]}
    elif len(list_text) > 2:
        if dict_count[list_text[0]] == 1:
            dict_description[str(i)] = {list_text[0]: {}}
            for j, text in enumerate(list_text[1:]):
                dict_description[str(i)][list_text[0]][str(j)] = text
        else:
            if list_text[0] in dict_no:
                dict_no[list_text[0]] += 1
            else:
                dict_no[list_text[0]] = 1
            dict_description[str(i)] = {f'{list_text[0]}-{dict_no[list_text[0]]}': {}}
            for j, text in enumerate(list_text[1:]):
                dict_description[str(i)][f'{list_text[0]}-{dict_no[list_text[0]]}'][str(j)] = text
    i += 1

for j, specification in enumerate(list_specification):
    key = specification.xpath('./text()')[0].strip()
    if dict_count[key] == 1:
        dict_description[str(i)] = {key: html.xpath('//div[@id="product-description-tab"]/specifications/value')[j].xpath('./text()')[0].strip()}
    else:
        if key in dict_no:
            dict_no[key] += 1
        else:
            dict_no[key] = 1
        dict_description[str(i)] = {f'{key}-{dict_no[key]}': html.xpath('//div[@id="product-description-tab"]/specifications/value')[j].xpath('./text()')[0].strip()}
    i += 1

for p in list_p:
    if p.xpath('./strong/text()'):
        key = p.xpath('./strong/text()')[0].strip().removesuffix(':').strip()
        if key == 'Item Condition':
            if dict_count[key] == 1:
                dict_description[str(i)] = {key: p.xpath('./span/text()')[0].strip()}
            else:
                if key in dict_no:
                    dict_no[key] += 1
                else:
                    dict_no[key] = 1
                dict_description[str(i)] = {f'{key}-{dict_no[key]}': p.xpath('./span/text()')[0].strip()}
            i += 1
        elif key == 'WARNING':
            if dict_count[key] == 1:
                dict_description[str(i)] = {key: p.xpath('./text()')[0].strip()}
            else:
                if key in dict_no:
                    dict_no[key] += 1
                else:
                    dict_no[key] = 1
                dict_description[str(i)] = {f'{key}-{dict_no[key]}': p.xpath('./text()')[0].strip()}
            i += 1
        
dict_description

{'0': {'Quantity': '2 Piece Set'},
 '6': {'Item Condition': 'New'},
 '7': {'WARNING': 'This product can expose you to chemicals including Mercaptobenzothiazole, which is known to the State of California to cause cancer, and birth defects or other reproductive harm.\xa0 For more information, go to www.P65Warnings.ca.gov.'}}

In [13]:
import json

dict_ = json.loads(resp.text.split('<!-- Google structured data  Markup -->')[1].split('<script type="application/ld+json">')[1].split('</script>')[0])

dict_

{'@context': 'https://schema.org/',
 '@type': 'Product',
 'name': '1AWSR00095-1978-81 Chevrolet Pontiac Roofrail Weatherstrip Seal 2 Piece Set Metro Moulded Parts RR 5020',
 'image': ['//cdn.1aauto.com/partimage/WSR/1AWSR00095/2187fb6f954c4e6cb9c4287963975d64_490.jpg'],
 'description': '<div class="item_specifications">\n <p>\n  These weatherstrips are some of the highest quality reproduction available. This weatherstrip seals the door glass to the body of the vehicle.\n </p>\n <p>\n  <strong>Specifications</strong>\n </p>\n <ul>\n  <li>Comes as a <strong>PAIR</strong>\n   <ul>\n    <li>Fits both LH (Driver Side) &amp; RH (Passenger Side) \n    </li>\n   </ul>\n  </li>\n  <li>Molded exactly like the original on your car\n  </li>\n  <li>Smooth soft fine grain rubber\n  </li>\n  <li>Direct Replacement\n  </li>\n  <li>\n   <strong>Correct for 2 Door models only</strong>\n  </li>\n </ul>\n <p>\n  If your weatherstripping is causing leaks allowing damaging rain in, then this seal is for you

In [14]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Type': type_,
                         'SKU': dict_['mpn'].strip() if dict_['mpn'] else '',
                         'Part Number': dict_['sku'].strip(),
                         'Brand': dict_['brand']['name'].strip(),
                         'Tag': tag,
                         'Title': title,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': json.dumps(dict_src),
                         'OEM Url': f'''https://www.1aauto.com/catalog/product/fit?skuId={resp.text.split('window.productId =')[1].split(';')[0].strip()}&pathName={input_.loc[a, 'Url'].removeprefix('https://www.1aauto.com').replace('/', '%2F')}''',
                         'Json_Description': json.dumps(dict_description),
                         'Price': f'''{dict_['offers']['priceCurrency'].strip()} {dict_['offers']['price']}''',
                         'Rating': dict_['aggregateRating']['ratingValue'].strip() if dict_['aggregateRating'] else '',
                         'Rating Count': int(dict_['aggregateRating']['ratingCount'].strip()) if dict_['aggregateRating'] else 0}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Type,SKU,Part Number,Brand,Tag,Title,Picture,Url,Json_Src,Vehicle Url,Json_Description,Price,Rating,Rating Count
0,660,Weatherstripping,RR 5020,1AWSR00095,Metro Moulded Parts,Replaces,1978-81 Chevrolet Pontiac Roofrail Weatherstri...,,https://www.1aauto.com/1978-81-chevrolet-ponti...,"{""0"": ""https://cdn.1aauto.com/partimage/WSR/1A...",https://www.1aauto.com/catalog/product/fit?sku...,"{""0"": {""Quantity"": ""2 Piece Set""}, ""6"": {""Item...",USD 97.92,,0


In [15]:
crawler_status = 'ok'

crawler_status

'ok'

In [16]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [17]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Type,SKU,Part Number,Brand,Tag,Title,Picture,Url,Json_Src,Vehicle Url,Json_Description,Price,Rating,Rating Count
0,660,Weatherstripping,RR 5020,1AWSR00095,Metro Moulded Parts,Replaces,1978-81 Chevrolet Pontiac Roofrail Weatherstri...,,https://www.1aauto.com/1978-81-chevrolet-ponti...,"{""0"": ""https://cdn.1aauto.com/partimage/WSR/1A...",https://www.1aauto.com/catalog/product/fit?sku...,"{""0"": {""Quantity"": ""2 Piece Set""}, ""6"": {""Item...",USD 97.92,,0


In [18]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
